In [254]:
import requests
import pandas as pd


def read_data():
    """reads location data for each municipality and merges it with the ranking data"""
    df_safety = pd.read_csv('safety-processed.csv')

    df_coordinates = pd.read_csv('WGS84_koordinaten_2019.csv', encoding='ISO-8859-1', header=0, sep=';')
    df_coordinates = df_coordinates[df_coordinates['Kantonskürzel'] == 'LU']
    df_coordinates = df_coordinates.drop_duplicates(subset='Ortschaftsname', keep='first')

    df_coordinates['Ortschaftsname'] = df_coordinates['Ortschaftsname'].str.replace(' LU', '')
    df_coordinates['Ortschaftsname'] = df_coordinates['Ortschaftsname'].str.replace('Escholzmatt',
                                                                                    'Escholzmatt-Marbach')
    df_coordinates['Ortschaftsname'] = df_coordinates['Ortschaftsname'].str.replace(' b. Willisau', '')

    df_municipalities = pd.merge(df_safety, df_coordinates[['Ortschaftsname', 'E', 'N']], left_on='Gemeindename',
                                 right_on='Ortschaftsname', how='left')
    df_municipalities = df_municipalities.rename(columns={'E': 'Longitude', 'N': 'Latitude'})
    return df_municipalities

In [255]:
def coop_data(filename: str):
    """adds a column with the search text to the coop data"""
    df_coop = pd.read_csv( filename, header=0, sep=',')
    df_coop['searchText'] = df_coop['Adresse'] + ' ' + df_coop['PLZ'].astype(str) + ' ' + df_coop['Ort']
    return df_coop

In [256]:
df_municipality = read_data()

C:\Users\Fabio\AppData\Local\Temp\ipykernel_32480\790559599.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df_coordinates['Ortschaftsname'] = df_coordinates['Ortschaftsname'].str.replace(' b. Willisau', '')


In [257]:
df_coop = coop_data('coop-pronto.csv')

In [258]:
_API_URL = 'https://api3.geo.admin.ch/rest/services/api/SearchServer'

In [259]:
def search_api_to_json(searchText, layer):
    url = f"{_API_URL}?type=locations&searchText={searchText}&origins={layer}"
    return requests.get(url).json()

In [260]:
def get_lat_lon_from_api_json(json):
    return json['results'][0]['attrs']['lon'], json['results'][0]['attrs']['lat']

In [261]:
df_coop['Longitude'] = None
df_coop['Latitude'] = None

for i, searchText in df_coop['searchText'].items():
    search_result = search_api_to_json(searchText, 'address')
    if search_result['results']:
        longitude, latitude = get_lat_lon_from_api_json(search_result)
        df_coop.loc[i, 'Longitude'] = longitude
        df_coop.loc[i, 'Latitude'] = latitude

df_coop

,Filialname,Adresse,PLZ,Ort,searchText,Longitude,Latitude
0,Coop Pronto Adligenswil,Luzernerstrasse 7,6043,Adligenswil,Luzernerstrasse 7 6043 Adligenswil,8.3576,47.063889
1,Coop Pronto Ebikon,Luzernerstrasse 22,6030,Ebikon,Luzernerstrasse 22 6030 Ebikon,8.334073,47.077675
2,Coop Pronto Schüpfheim,Hindervormüli 15,6170,Schüpfheim,Hindervormüli 15 6170 Schüpfheim,8.00747,46.945309
3,Coop Pronto Emmenbrücke,Rothenburgstrasse 24,6020,Emmenbrücke,Rothenburgstrasse 24 6020 Emmenbrücke,8.272595,47.084312
4,Coop Pronto Emmenbrücke,Seetalstrasse 44,6020,Emmenbrücke,Seetalstrasse 44 6020 Emmenbrücke,8.291343,47.074116
5,Coop Pronto Horw,Kantonsstrasse 1,6048,Horw,Kantonsstrasse 1 6048 Horw,8.305859,47.025272
6,Coop Pronto Kriens,Obernauerstrasse 3,6010,Kriens,Obernauerstrasse 3 6010 Kriens,8.268352,47.033665
7,Coop Pronto Luzern Bahnhof,Bahnhofplatz 2,6003,Luzern,Bahnhofplatz 2 6003 Luzern,8.309069,47.050335
8,Coop Pronto Luzern Brünigstrasse,Brünigstrasse 7,6005,Luzern,Brünigstrasse 7 6005 Luzern,8.312065,47.04324
9,Coop Pronto Luzern Pilatusstrasse,Pilatusstrasse 20,6003,Luzern,Pilatusstrasse 20 6003 Luzern,8.305802,47.048717


In [262]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Berechnet die Entfernung zwischen zwei geographischen Koordinaten
    in Kilometern mit der Haversine-Formel.
    """
    from math import radians, cos, sin, asin, sqrt
    # Konvertieren in Radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # Haversine-Formel
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    # Radius der Erde (in Kilometern)
    r = 6371
    # Berechnung der Entfernung
    return c * r

In [263]:
for i, row in df_municipality.iterrows():
    distances = []
    for j, coop_row in df_coop.iterrows():
        distance = haversine_distance(row['Latitude'], row['Longitude'], coop_row['Latitude'], coop_row['Longitude'])
        distances.append(distance)
    df_municipality.at[i, 'Distanz zu nächsten Prontoshop in km'] = min(distances)


# Spalte 'Score' soll die erste Spalte sein Score ist im Moment die 5. letzte Spalte

cols = list(df_municipality.columns)
score_index = cols.index('Score')
cols = [cols[score_index]] + cols[:score_index] + cols[score_index+1:]

df_municipality = df_municipality.reindex(columns=cols)

# schreibe in ein csv names asn2.csv
df_municipality.to_csv('asn2.csv')

df_municipality

,Score,Gemeindename,Total Anzahl Personen Sozialhilfe,Total Anzahl Dossiers,Total Sozialhilfequote,Total Sozialhilfequote Schweizer,Total Sozialhilfequote Ausländer,Alter 0-17,Alter 18-64,Alter 65+,...,davon verletze Personen,Lenker/innen,Mitfahrer/innen,Fussgänger/innen,Fahrzeugähnliche Geräte,Rank,Ortschaftsname,Longitude,Latitude,Distanz zu nächsten Prontoshop in km
0,16.75,Adligenswil,42.0,29.0,0.8,0.4,3.6,1.2,0.9,0.2,...,3,3,0,0,0,27.0,Adligenswil,8.365,47.071,0.969228
1,7.25,Aesch,11.0,10.0,0.9,0.9,0.6,0.5,1.2,0.0,...,4,3,1,0,0,10.0,Aesch,8.239,47.255,10.666998
2,9.75,Alberswil,9.0,5.0,1.4,1.0,3.3,2.6,1.3,0.0,...,4,4,0,0,0,16.0,Alberswil,7.997,47.150,6.701790
3,10.00,Altbüron,25.0,12.0,2.5,2.1,2.7,6.1,2.0,0.0,...,1,1,0,0,0,17.0,Altbüron,7.882,47.180,8.440348
4,14.00,Altishofen,25.0,17.0,1.3,0.5,5.3,2.1,1.3,0.0,...,5,4,1,0,0,25.0,Altishofen,7.960,47.202,2.059905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1.25,Honau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,0,0,2.0,Honau,8.406,47.133,2.679949
76,20.75,Rain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17,11,4,2,0,31.0,Rain,8.263,47.130,4.455348
77,0.00,Romoos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1.0,Romoos,8.024,47.011,6.700320
78,20.75,Schenkon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11,8,1,2,0,31.0,Schenkon,8.136,47.173,2.413216
